In [142]:
import sqlite3
import json
import datetime

# Dades per defecte
default_data = [
    ["7080", "Fundació Catalana per a la Recerca i la Innovació (FCRi)",
     "ADMINISTRACIÓ DE LA GENERALITAT DE CATALUNYA", "PARTICIPACIÓ GENERALITAT I PÚBLICA",
     "Administració de la Generalitat de Catalunya", "", "Si", "37,50", "40,00", "", "0,00"],
    ["7080", "Fundació Catalana per a la Recerca i la Innovació (FCRi)",
     "SECTOR PRIVAT", "CAP DELS DOS", "Banco Bilbao Vizcaya Argentaria", "", "No", "6,25", "6,67", "", "0,00"],
    ["7080", "Fundació Catalana per a la Recerca i la Innovació (FCRi)",
     "SECTOR PRIVAT", "CAP DELS DOS", "Naturgy Energy Group S.A.", "BAIXA", "No", "6,25", "0,00", "", "0,00"],
]

origHeaders = [
    "Codi Catàleg", "Denominació", "Categorització partícip", "Sectorització del partícip",
    "Denominació partícip (agregat)", "Codi Catàleg", "P. Física", "Vincle primari",
    "Percentatge participació", "Clàusules particulars", "Percentatge del cap. Social (mercantils)"
]


In [143]:
# ---------------------------------------------
# DBHandler: Gestor de la base de dades amb SQLite
# ---------------------------------------------
class DBHandler:
    def __init__(self, db_name='dades.db'):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.create_tables()

    def create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS dades (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                fila INTEGER,
                columna INTEGER,
                valor TEXT
            )
        ''')
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS historial (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                fila INTEGER,
                columna INTEGER,
                valor_antic TEXT,
                valor_nou TEXT,
                data_canvi TEXT,
                documents TEXT
            )
        ''')
        self.conn.commit()

    def init_data(self, default_data):
        self.cursor.execute("SELECT COUNT(*) FROM dades")
        count = self.cursor.fetchone()[0]
        if count == 0:
            for fila, row in enumerate(default_data):
                for columna, valor in enumerate(row):
                    self.guarda_dada(fila, columna, valor)

    def guarda_dada(self, fila, columna, valor):
        self.cursor.execute('INSERT INTO dades (fila, columna, valor) VALUES (?, ?, ?)',
                            (fila, columna, valor))
        self.conn.commit()

    def guarda_canvi(self, fila, columna, valor_antic, valor_nou, documents):
        data_canvi = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        docs_json = json.dumps(documents)
        self.cursor.execute('''
            INSERT INTO historial (fila, columna, valor_antic, valor_nou, data_canvi, documents)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (fila, columna, valor_antic, valor_nou, data_canvi, docs_json))
        self.conn.commit()

    def get_all_dades(self):
        self.cursor.execute("SELECT fila, columna, valor FROM dades")
        result = self.cursor.fetchall()
        if not result:
            return None
        rows = {}
        for fila, columna, valor in result:
            rows.setdefault(fila, {})[columna] = valor
        data_loaded = []
        max_row = max(rows.keys()) if rows else 0
        max_col = max(max(r.keys()) for r in rows.values()) if rows else 0
        for r in range(max_row+1):
            row_data = []
            for c in range(max_col+1):
                row_data.append(rows.get(r, {}).get(c, ""))
            data_loaded.append(row_data)
        return data_loaded

    def get_all_historial(self):
        self.cursor.execute("SELECT fila, columna, valor_antic, valor_nou, data_canvi, documents FROM historial")
        result = self.cursor.fetchall()
        hist = {}
        for fila, columna, valor_antic, valor_nou, data_canvi, docs in result:
            key = f"{fila}-{columna}"
            try:
                docs_list = json.loads(docs)
            except:
                docs_list = []
            record = {"change": f"{valor_antic} → {valor_nou}",
                      "date": data_canvi,
                      "documents": docs_list}
            hist.setdefault(key, []).append(record)
        return hist

    def close(self):
        self.conn.close()



In [144]:
# ---------------------------------------------
# DataModel: Gestiona les dades i l'historial utilitzant DBHandler
# ---------------------------------------------
class DataModel:
    def __init__(self):
        self.db_handler = DBHandler()
        self.db_handler.init_data(default_data)
        self.data = self.db_handler.get_all_dades() or default_data
        self.history = self.db_handler.get_all_historial() or {}

    def guardaCanvi(self, row, col, oldText, newText, documents):
        self.db_handler.guarda_canvi(row, col, oldText, newText, documents)
        self.db_handler.guarda_dada(row, col, newText)
        record = {"change": f"{oldText} → {newText}",
                  "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                  "documents": documents.copy()}
        key = f"{row}-{col}"
        self.history.setdefault(key, []).append(record)


In [145]:
from PyQt5.QtWidgets import (QWidget, QVBoxLayout, QHBoxLayout, QTableWidget, QTableWidgetItem, 
                             QPushButton, QLineEdit, QCheckBox, QDialog, QLabel, QListWidget, 
                             QFileDialog, QMenu, QColorDialog, QInputDialog, QMessageBox)
from PyQt5.QtCore import Qt, pyqtSignal
from PyQt5.QtGui import QBrush, QColor
import datetime
import os

In [146]:
# ---------------------------------------------
# EditableCellWidget: Widget editable per cada cel·la
# ---------------------------------------------
class EditableCellWidget(QWidget):
    selectionChanged = pyqtSignal(str, bool)
    def __init__(self, cell_id, parent=None):
        super().__init__(parent)
        self.cell_id = cell_id
        self.lineEdit = QLineEdit()
        self.lineEdit.setStyleSheet("color: red; background-color: #ffcccc;")
        self.lineEdit.setContextMenuPolicy(Qt.NoContextMenu)
        self.checkBox = QCheckBox()
        self.checkBox.setEnabled(False)
        layout = QHBoxLayout()
        layout.addWidget(self.lineEdit)
        layout.addWidget(self.checkBox)
        layout.setContentsMargins(0, 0, 0, 0)
        self.setLayout(layout)
        self.lineEdit.textChanged.connect(self.onTextChanged)
        self.checkBox.stateChanged.connect(self.onCheckboxChanged)

    def onTextChanged(self, text):
        if text.strip():
            self.checkBox.setEnabled(True)
        else:
            self.checkBox.setEnabled(False)
            self.checkBox.setChecked(False)
        parentTable = self.getParentTable()
        if parentTable:
            row = int(self.cell_id.split('-')[0])
            parentTable.updateAggregateColorForRow(row)

    def onCheckboxChanged(self, state):
        self.selectionChanged.emit(self.cell_id, state == Qt.Checked)

    def getParentTable(self):
        widget = self
        while widget:
            if hasattr(widget, "table"):
                return widget
            widget = widget.parent()
        return None

In [147]:
# ---------------------------------------------
# EditableTableView: Vista de la taula editable
# ---------------------------------------------
class EditableTableView(QWidget):
    def __init__(self, dataModel):
        super().__init__()
        self.dataModel = dataModel
        self.pendingDocuments = {}   # Document pendents per cel·la
        self.selectedChanges = set()   # Conjunt de cell_id seleccionats
        self.originalData = {}
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()
        # Botons superiors
        btnLayout = QHBoxLayout()
        self.toggleBtn = QPushButton("Veure Modificacions")
        self.validateBtn = QPushButton("Validar Canvis")
        self.associateDocsBtn = QPushButton("Associar documentació als canvis pendents")
        self.showFullHistoryBtn = QPushButton("Mostrar Historial Complet")
        self.historicalBtn = QPushButton("Situació en una data")
        for btn in [self.toggleBtn, self.validateBtn, self.associateDocsBtn, self.showFullHistoryBtn, self.historicalBtn]:
            btnLayout.addWidget(btn)
        layout.addLayout(btnLayout)
        
        # Creació de la taula
        numRows = len(self.dataModel.data)
        numOrigCols = len(self.dataModel.data[0])
        totalCols = numOrigCols * 2
        self.table = QTableWidget(numRows, totalCols)
        headers = []
        for h in origHeaders:
            headers.extend([h, "Editar " + h])
        self.table.setHorizontalHeaderLabels(headers)
        for row in range(numRows):
            for col in range(numOrigCols):
                value = self.dataModel.data[row][col]
                item = QTableWidgetItem(value)
                item.setForeground(QColor("black"))
                item.setFlags(item.flags() & ~Qt.ItemIsEditable)
                self.table.setItem(row, col*2, item)
                self.originalData[(row, col)] = value
                cell_id = f"{row}-{col}"
                widget = EditableCellWidget(cell_id)
                widget.selectionChanged.connect(self.onSelectionChanged)
                self.table.setCellWidget(row, col*2+1, widget)
        # Amagar columnes d'edició
        for col in range(totalCols):
            if col % 2 == 1:
                self.table.setColumnHidden(col, True)
        self.table.setContextMenuPolicy(Qt.CustomContextMenu)
        self.table.customContextMenuRequested.connect(self.onContextMenu)
        layout.addWidget(self.table)
        self.setLayout(layout)
        
        # Connexions dels botons
        self.toggleBtn.clicked.connect(lambda: print("ToggleBotó clicat"))

        self.validateBtn.clicked.connect(self.validateChanges)
        self.associateDocsBtn.clicked.connect(self.associateDocumentation)
        self.showFullHistoryBtn.clicked.connect(self.showFullHistory)
        self.historicalBtn.clicked.connect(self.showHistoricalSituation)
        
        for row in range(self.table.rowCount()):
            self.updateAggregateColorForRow(row)
        self.clearAllSpans()
        self.groupCells(list(range(self.table.rowCount())), 0)

    def clearAllSpans(self):
        for row in range(self.table.rowCount()):
            for col in range(self.table.columnCount()):
                self.table.setSpan(row, col, 1, 1)

    def groupCells(self, rows, origCol):
        numOrigCols = len(self.dataModel.data[0])
        if origCol >= numOrigCols or not rows:
            return
        table_col = origCol * 2
        groups = []
        current_group = [rows[0]]
        for r in rows[1:]:
            curr_item = self.table.item(r, table_col)
            prev_item = self.table.item(current_group[-1], table_col)
            if curr_item and prev_item and curr_item.text() == prev_item.text():
                current_group.append(r)
            else:
                groups.append(current_group)
                current_group = [r]
        if current_group:
            groups.append(current_group)
        for group in groups:
            if len(group) > 1:
                self.table.setSpan(group[0], table_col, len(group), 1)
            self.groupCells(group, origCol+1)

    def toggleModificacions(self):
        print("toggleModificacions cridat")
        totalCols = self.table.columnCount()
        hidden = self.table.isColumnHidden(1)
        print("La columna 1 està amagada?", hidden)
        newState = not hidden
        for col in range(totalCols):
            if col % 2 == 1:
                self.table.setColumnHidden(col, not newState)
                print("Columna", col, "configurada com a", "visible" if newState else "amagada")
        for btn in [self.validateBtn, self.associateDocsBtn, self.showFullHistoryBtn]:
            btn.setVisible(newState)
            print("El botó", btn.text(), "ara està", "visible" if newState else "amagat")
        new_text = "Ocultar Modificacions" if newState else "Veure Modificacions"
        self.toggleBtn.setText(new_text)
        print("El text del botó toggle és ara:", new_text)


    def validateChanges(self):
        numOrigCols = len(self.dataModel.data[0])
        for row in range(self.table.rowCount()):
            for col in range(numOrigCols):
                editCol = col * 2 + 1
                widget = self.table.cellWidget(row, editCol)
                if widget:
                    newText = widget.lineEdit.text().strip()
                    if newText and widget.checkBox.isChecked():
                        origItem = self.table.item(row, col*2)
                        oldText = origItem.text() if origItem else ""
                        if origItem:
                            origItem.setText(newText)
                        else:
                            new_item = QTableWidgetItem(newText)
                            new_item.setForeground(QColor("black"))
                            self.table.setItem(row, col*2, new_item)
                        cell_id = f"{row}-{col}"
                        self.dataModel.db_handler.guarda_canvi(row, col, oldText, newText,
                                                               self.pendingDocuments.get(cell_id, []))
                        self.dataModel.db_handler.guarda_dada(row, col, newText)
                        record = {"change": f"{oldText} → {newText}",
                                  "date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                  "documents": self.pendingDocuments.get(cell_id, []).copy()}
                        self.dataModel.history.setdefault(cell_id, []).append(record)
                        self.pendingDocuments[cell_id] = []
                        widget.lineEdit.clear()
                        widget.checkBox.setChecked(False)
                        widget.checkBox.setEnabled(False)
                        self.selectedChanges.discard(cell_id)
        for row in range(self.table.rowCount()):
            self.updateAggregateColorForRow(row)
        self.reorderTable()

    def reorderTable(self):
        self.reorderData()
        numRows = self.table.rowCount()
        numOrigCols = len(self.dataModel.data[0])
        for row in range(numRows):
            for col in range(numOrigCols):
                value = self.dataModel.data[row][col]
                item = self.table.item(row, col*2)
                if item is None:
                    item = QTableWidgetItem(value)
                    item.setForeground(QColor("black"))
                    item.setFlags(item.flags() & ~Qt.ItemIsEditable)
                    self.table.setItem(row, col*2, item)
                else:
                    item.setText(value)
        self.clearAllSpans()
        self.groupCells(list(range(numRows)), 0)

    def reorderData(self):
        numRows = self.table.rowCount()
        numOrigCols = len(self.dataModel.data[0])
        rows_data = []
        for row in range(numRows):
            row_vals = []
            for col in range(numOrigCols):
                item = self.table.item(row, col*2)
                row_vals.append(item.text() if item else "")
            rows_data.append((row, row_vals))
        rows_data.sort(key=lambda x: tuple(x[1]))
        new_data = []
        new_originalData = {}
        for new_row, (old_row, values) in enumerate(rows_data):
            new_data.append(values)
            for col in range(numOrigCols):
                new_originalData[(new_row, col)] = values[col]
        self.dataModel.data = new_data
        self.originalData = new_originalData

    def updateAggregateColorForRow(self, row):
        numOrigCols = len(self.dataModel.data[0])
        pending = any(self.table.cellWidget(row, col*2+1) and 
                      self.table.cellWidget(row, col*2+1).lineEdit.text().strip() != ""
                      for col in range(numOrigCols))
        aggregate_col = 4 * 2
        item = self.table.item(row, aggregate_col) or QTableWidgetItem("")
        item.setBackground(QBrush(QColor("#D2B48C") if pending else QColor("green")))
        self.table.setItem(row, aggregate_col, item)

    def getCatalogCodeFromCell(self, cell_id):
        try:
            row, _ = map(int, cell_id.split('-'))
        except Exception:
            return None
        item = self.table.item(row, 0)
        return item.text().strip() if item else None

    # Aquest mètode llegeix el codi catàleg de la fila corresponent i obre la carpeta
    def openFolderForSelectedCell(self, cell_id):
        try:
            row, _ = map(int, cell_id.split('-'))
        except Exception:
            QMessageBox.critical(self, "Error", "Identificador de cel·la invàlid.")
            return
        item = self.table.item(row, 0)
        if item is None:
            QMessageBox.critical(self, "Error", "No s'ha trobat el codi catàleg en aquesta fila.")
            return
        catalog_code = item.text().strip()
        base_folder = r"D:\34761933D\Desktop\NOU CatEns\UI - Partícips - OGS - Persones\estil excel\Documents"
        folder_path = os.path.join(base_folder, catalog_code)
        if os.path.exists(folder_path):
            os.startfile(folder_path)
        else:
            QMessageBox.information(self, "Informació", f"La carpeta per al codi '{catalog_code}' no existeix.")

    def associateDocumentation(self):
        print("associateDocumentation cridat")
        print("selectedChanges =", self.selectedChanges)
        if not self.selectedChanges:
            QMessageBox.information(self, "Informació", "No hi ha canvis seleccionats.")
            return
        if len(self.selectedChanges) != 1:
            QMessageBox.information(self, "Informació", "Si us plau, selecciona una sola fila per associar documentació.")
            return
        cell_id = next(iter(self.selectedChanges))
        print("Utilitzant cell_id:", cell_id)
        self.openDocumentFolderForCell(cell_id)
    
    def openDocumentFolderForSelectedCell(self, cell_id):
        try:
            row, _ = map(int, cell_id.split('-'))
        except Exception as e:
            QMessageBox.critical(self, "Error", "Identificador de cel·la invàlid.")
            print("Error en convertir cell_id:", cell_id, e)
            return
        item = self.table.item(row, 0)
        if item is None:
            QMessageBox.critical(self, "Error", "No s'ha trobat el codi catàleg en aquesta fila.")
            print("No s'ha trobat l'item a la fila", row)
            return
        catalog_code = item.text().strip()
        print("Codi catàleg llegit:", catalog_code)
        base_folder = r"D:\34761933D\Desktop\NOU CatEns\UI - Partícips - OGS - Persones\estil excel\Documents"
        folder_path = os.path.join(base_folder, catalog_code)
        print("Ruta de la carpeta:", folder_path)
        if os.path.exists(folder_path):
            print("La carpeta existeix, s'obre...")
            os.startfile(folder_path)
        else:
            QMessageBox.information(self, "Informació", f"La carpeta per al codi '{catalog_code}' no existeix.")
            print("La carpeta no existeix:", folder_path)



    def showFullHistory(self):
        dialog = QDialog(self)
        dialog.setWindowTitle("Historial Complet de Canvis")
        layout = QVBoxLayout()
        if not self.dataModel.history:
            layout.addWidget(QLabel("No hi ha canvis registrats."))
        else:
            for cid, recs in self.dataModel.history.items():
                layout.addWidget(QLabel(f"Camp {cid}:"))
                for rec in recs[::-1]:
                    layout.addWidget(QLabel(f"{rec['date']}: {rec['change']}"))
                    for doc in rec.get("documents", []):
                        layout.addWidget(QLabel(f"   Document: {doc['name']}"))
        extBtn = QPushButton("Afegir Document Extern")
        extBtn.clicked.connect(self.openDocumentManager)
        layout.addWidget(extBtn)
        dialog.setLayout(layout)
        dialog.exec_()

    def showHistoricalSituation(self):
        dt_str, ok = QInputDialog.getText(self, "Situació en una data",
                                          "Introdueix la data i hora (YYYY-MM-DD HH:MM:SS):")
        if not ok or not dt_str.strip():
            return
        try:
            chosen_dt = datetime.datetime.strptime(dt_str.strip(), "%Y-%m-%d %H:%M:%S")
        except ValueError:
            QMessageBox.critical(self, "Error", "Format de data invàlid (YYYY-MM-DD HH:MM:SS).")
            return
        numRows = self.table.rowCount()
        numOrigCols = len(self.dataModel.data[0])
        totalCols = numOrigCols * 2
        dialog = QDialog(self)
        dialog.setWindowTitle("Situació en una data: " + dt_str)
        dlayout = QVBoxLayout()
        queryTable = QTableWidget(numRows, totalCols)
        headers = [f"{h} (a la data)" for h in origHeaders] + [f"{h} (posterior)" for h in origHeaders]
        queryTable.setHorizontalHeaderLabels(headers)
        for row in range(numRows):
            for col in range(numOrigCols):
                key = f"{row}-{col}"
                initial = self.originalData[(row, col)]
                state, next_val, docs_state, docs_next = initial, "", [], []
                if key in self.dataModel.history:
                    recs = sorted(self.dataModel.history[key],
                                  key=lambda r: datetime.datetime.strptime(r["date"], "%Y-%m-%d %H:%M:%S"))
                    for r in recs:
                        r_dt = datetime.datetime.strptime(r["date"], "%Y-%m-%d %H:%M:%S")
                        if r_dt <= chosen_dt:
                            state = r["change"].split("→")[-1].strip()
                            docs_state = r.get("documents", [])
                        elif r_dt > chosen_dt and not next_val:
                            next_val = r["change"].split("→")[-1].strip()
                            docs_next = r.get("documents", [])
                state_disp = "Sense canvis" if state == initial and not next_val else state
                next_disp = "Sense canvis" if state == initial and not next_val else (next_val or "No hi ha")
                bg_state = "white" if state == initial and not next_val else "#ffcccc"
                bg_next = "white" if state == initial and not next_val else "#ccffcc"
                item_state = QTableWidgetItem(state_disp)
                item_next = QTableWidgetItem(next_disp)
                item_state.setFlags(item_state.flags() & ~Qt.ItemIsEditable)
                item_next.setFlags(item_next.flags() & ~Qt.ItemIsEditable)
                item_state.setBackground(QBrush(QColor(bg_state)))
                item_next.setBackground(QBrush(QColor(bg_next)))
                item_state.setData(Qt.UserRole, docs_state)
                item_next.setData(Qt.UserRole, docs_next)
                queryTable.setItem(row, col*2, item_state)
                queryTable.setItem(row, col*2+1, item_next)
        queryTable.resizeRowsToContents()
        queryTable.setContextMenuPolicy(Qt.CustomContextMenu)
        queryTable.customContextMenuRequested.connect(lambda point: self._onQueryContextMenu(queryTable, point))
        dlayout.addWidget(queryTable)
        dialog.setLayout(dlayout)
        dialog.exec_()

    def _onQueryContextMenu(self, table, point):
        item = table.itemAt(point)
        menu = QMenu(self)
        if item:
            docs = item.data(Qt.UserRole)
            if docs:
                for doc in docs:
                    menu.addAction(doc["name"])
            else:
                menu.addAction("No hi ha documents associats.")
        menu.exec_(table.mapToGlobal(point))

    def onContextMenu(self, pos):
        index = self.table.indexAt(pos)
        if not index.isValid():
            return
        row = index.row()
        col = index.column()
        cell_id = f"{row}-{(col-1)//2}" if col % 2 == 1 else f"{row}-{col//2}"
        isEditable = (col % 2 == 1)
        menu = QMenu(self)
        actionTextColor = menu.addAction("Tria color de la lletra")
        actionBgColor = menu.addAction("Tria color de fons")
        actionHistory = menu.addAction("Mostrar Historial / Documentació")
        action = menu.exec_(self.table.viewport().mapToGlobal(pos))
        if action == actionTextColor:
            self.changeCellTextColor(cell_id)
        elif action == actionBgColor:
            self.changeCellBackgroundColor(cell_id)
        elif action == actionHistory:
            self.showHistoryDialog(cell_id, isEditable)

    def addDocumentToCell(self, cell_id, parentDialog=None):
        filePath, _ = QFileDialog.getOpenFileName(self, "Selecciona Document", "", "All Files (*)")
        if filePath:
            fileName = os.path.basename(filePath)
            doc = {"path": filePath, "name": fileName}
            self.pendingDocuments.setdefault(cell_id, []).append(doc)
            if parentDialog:
                parentDialog.close()

    def changeCellTextColor(self, cell_id):
        row, col = map(int, cell_id.split('-'))
        item = self.table.item(row, col*2)
        if item:
            color = QColorDialog.getColor()
            if color.isValid():
                item.setForeground(color)

    def changeCellBackgroundColor(self, cell_id):
        row, col = map(int, cell_id.split('-'))
        item = self.table.item(row, col*2)
        if item:
            color = QColorDialog.getColor()
            if color.isValid():
                item.setBackground(QBrush(color))

    def onSelectionChanged(self, cell_id, selected):
        if selected:
            self.selectedChanges.add(cell_id)
        else:
            self.selectedChanges.discard(cell_id)

    def openDocumentManager(self):
        entity_code, ok = QInputDialog.getText(self, "Entitat", "Introdueix el número d'entitat:")
        if ok and entity_code.strip():
            from DocumentManagerDialog import DocumentManagerDialog  # Assegura't d'importar correctament
            dm = DocumentManagerDialog(entity_code.strip(), self)
            dm.exec_()

    def openDocumentFolderForCell(self, cell_id):
        try:
            row, _ = map(int, cell_id.split('-'))
        except Exception:
            QMessageBox.critical(self, "Error", "Identificador de cel·la invàlid.")
            return
        item = self.table.item(row, 0)
        if item is None:
            QMessageBox.critical(self, "Error", "No s'ha trobat el codi catàleg.")
            return
        codi = item.text().strip()
        base_folder = r"D:\34761933D\Desktop\NOU CatEns\UI - Partícips - OGS - Persones\estil excel\Documents"
        folder_path = os.path.join(base_folder, codi)
        if os.path.exists(folder_path):
            os.startfile(folder_path)
        else:
            QMessageBox.information(self, "Informació", "Aquesta entitat no té documents.")

    # Versió modificada: en clicar "Associar documentació als canvis pendents", 
    # es llegeix el valor del "Codi Catàleg" de la mateixa línia (una única selecció)
    # i s'obre la carpeta corresponent.
    def associateDocumentation(self):
        if not self.selectedChanges:
            QMessageBox.information(self, "Informació", "No hi ha canvis seleccionats.")
            return
        if len(self.selectedChanges) != 1:
            QMessageBox.information(self, "Informació", "Si us plau, selecciona una sola fila per associar documentació.")
            return
        cell_id = next(iter(self.selectedChanges))
        self.openDocumentFolderForCell(cell_id)


In [148]:
# ---------------------------------------------
# DocumentManagerDialog: Exemple de gestor documental extern
# ---------------------------------------------
class DocumentManagerDialog(QDialog):
    def __init__(self, entity_code=None, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Gestor Documental - Entitat: " + entity_code)
        self.resize(500,700)
        layout = QVBoxLayout()
        layout.addWidget(QLabel("Interfície del gestor documental..."))
        self.setLayout(layout)

In [149]:
%gui qt5
window = EditableTable()
window.show()


ToggleBotó clicat
ToggleBotó clicat
ToggleBotó clicat
ToggleBotó clicat
